In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from skimage.feature import hog
from data_loading import load_data
from ANN import ANN

# Correct paths to your dataset
csv_file = r"./archive/data.csv"
base_folder = r"./archive/dataset"

# Load images and labels
X, y = load_data(csv_file, base_folder)

### Feature Engineering (same as before) ###
def rgb_to_grayscale(images):
    return np.dot(images[..., :3], [0.2989, 0.5870, 0.1140])

def extract_hog_features(images):
    hog_features = []
    for img in images:
        features = hog(img, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)
        hog_features.append(features)
    return np.array(hog_features)

def apply_pca(features, n_components=100):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(features)

def normalize_features(features):
    scaler = StandardScaler()
    return scaler.fit_transform(features)

import numpy as np

def one_hot_encode(labels, num_classes):
    one_hot = np.zeros((labels.size, num_classes))  # Initialize a zero matrix
    one_hot[np.arange(labels.size), labels] = 1     # Set the appropriate indices to 1
    return one_hot


# Preprocess features
X_gray = rgb_to_grayscale(X)
X_hog = extract_hog_features(X_gray)
X_pca = apply_pca(X_hog, n_components=100)
X_normalized = normalize_features(X_pca)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# One-hot encode labels for ANN
y_one_hot = one_hot_encode(y_encoded, 5)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_one_hot, test_size=0.2, random_state=42)

### Train ANN Model ###
input_size = X_normalized.shape[1]
hidden_size = 128
output_size = y_one_hot.shape[1]

ann_model = ANN(input_size, hidden_size, output_size, learning_rate=0.01, epochs=1000)
ann_model.train(X_train, y_train)

### Test ANN Model ###
y_test_pred = ann_model.predict(X_test)
test_accuracy = np.mean(np.argmax(y_test, axis=1) == y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

In [22]:
### Train ANN Model ###
input_size = X_normalized.shape[1]
hidden_size = 128
output_size = y_one_hot.shape[1]

ann_model = ANN(input_size, hidden_size, output_size, learning_rate=0.2, epochs=1000)
ann_model.train(X_train, y_train)

### Test ANN Model ###
y_test_pred = ann_model.predict(X_test)
test_accuracy = np.mean(np.argmax(y_test, axis=1) == y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

Epoch 0, Loss: 1.6090544427929978
Epoch 100, Loss: 1.1275105927648845
Epoch 200, Loss: 0.9506268175639765
Epoch 300, Loss: 0.8800894375806178
Epoch 400, Loss: 0.8303845880890773
Epoch 500, Loss: 0.7786031108686623
Epoch 600, Loss: 0.725426376608246
Epoch 700, Loss: 0.6724968872835336
Epoch 800, Loss: 0.6209458135886696
Epoch 900, Loss: 0.5714262561502951
Test Accuracy: 0.6508771929824562
